Vamos trabalhar com uma base de dados sobre o pagamento de empréstimos fornecido a clientes por um banco. Nessa base a Meu objetivo é prever se os clientes pagarão ou não a dívida (variável default), de acordo com os seguintes atributos: renda (income), idade (age) e montante da dívida (loan). Clientes sinalizados com valor 0 para a variável target pagaram sua dívida e clientes com valor 1 são devedores na base histórica de dados.

**Etapas realizadas neste notebook:**

1.Limpeza dos dados e tratamento de valores faltantes <br>
2.Separação das bases com variáveis preditoras e target <br>
3.Transformação dos dados (escalonamento) <br>
4.Separação das bases em treino e teste <br>
5.Exportação das bases para serem usadas nos scripts com os algoritmos

----

### 1.Limpeza dos dados

In [43]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [66]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import pickle

Importando os dados:

In [45]:
credit_data = pd.read_csv("credit_data.csv")
credit_data.head()

,clientid,income,age,loan,default
0,1,66155.925095,59.017015,8106.532131,0
1,2,34415.153966,48.117153,6564.745018,0
2,3,57317.170063,63.108049,8020.953296,0
3,4,42709.534201,45.751972,6103.642260,0
4,5,66952.688845,18.584336,8770.099235,1


Inspeção geral dos dados:

In [46]:
# Contagem de valores nas classes
values, count = np.unique(credit_data["default"], return_counts=True)
[print(f'Valor: {v}, Contagem: {c}') for v,c in zip(values,count) ]

Valor: 0, Contagem: 1717
Valor: 1, Contagem: 283


[None, None]

In [47]:
credit_data.describe()

,clientid,income,age,loan,default
count,2000.000000,2000.000000,1997.000000,2000.000000,2000.000000
mean,1000.500000,45331.600018,40.807559,4444.369695,0.141500
std,577.494589,14326.327119,13.624469,3045.410024,0.348624
min,1.000000,20014.489470,-52.423280,1.377630,0.000000
25%,500.750000,32796.459717,28.990415,1939.708847,0.000000
50%,1000.500000,45789.117313,41.317159,3974.719419,0.000000
75%,1500.250000,57791.281668,52.587040,6432.410625,0.000000
max,2000.000000,69995.685578,63.971796,13766.051239,1.000000


In [48]:
credit_data.isnull().sum()

clientid    0
income      0
age         3
loan        0
default     0
dtype: int64

In [49]:
index = credit_data[credit_data["age"].isnull()].index
credit_data.iloc[index,:]

,clientid,income,age,loan,default
28,29,59417.805406,NaN,2082.625938,0
30,31,48528.852796,NaN,6155.784670,0
31,32,23526.302555,NaN,2862.010139,0


Percebemos que a coluna idade possui alguns problemas, como valores negativos e valores faltantes. Vamos explorar esses dados, descobrir onde estão esses registros e corrigir esses valores. Primeiro, substituir valores faltantes pelo valor da média desse atributo:

In [50]:
mean = credit_data[credit_data["age"] > 0]["age"].mean()
credit_data.fillna(mean,inplace=True)
credit_data.iloc[index,:]

,clientid,income,age,loan,default
28,29,59417.805406,40.9277,2082.625938,0
30,31,48528.852796,40.9277,6155.784670,0
31,32,23526.302555,40.9277,2862.010139,0


Agora vamos corrigir os registros que têm valores negativos, multiplicando por -1 para que se tornem positivos:

In [51]:
index = credit_data[credit_data["age"] < 0].index
credit_data.loc[index,"age"] = credit_data.loc[index,"age"]*(-1)
credit_data.loc[index,:] 

,clientid,income,age,loan,default
15,16,50501.726689,28.218361,3977.287432,0
21,22,32197.620701,52.423280,4244.057136,0
26,27,63287.038908,36.496976,9595.286289,0


Para que os dados de idade façam mais sentido para a vida real, vamos transformar também o tipo de dado para inteiro:

In [52]:
credit_data["age"] = credit_data["age"].astype(int)
credit_data.head()

,clientid,income,age,loan,default
0,1,66155.925095,59,8106.532131,0
1,2,34415.153966,48,6564.745018,0
2,3,57317.170063,63,8020.953296,0
3,4,42709.534201,45,6103.642260,0
4,5,66952.688845,18,8770.099235,1


Checando se os dados estão ajustados agora:

In [53]:
credit_data.describe()

,clientid,income,age,loan,default
count,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000
mean,1000.500000,45331.600018,40.425500,4444.369695,0.141500
std,577.494589,14326.327119,13.262594,3045.410024,0.348624
min,1.000000,20014.489470,18.000000,1.377630,0.000000
25%,500.750000,32796.459717,29.000000,1939.708847,0.000000
50%,1000.500000,45789.117313,41.000000,3974.719419,0.000000
75%,1500.250000,57791.281668,52.000000,6432.410625,0.000000
max,2000.000000,69995.685578,63.000000,13766.051239,1.000000


----

### 2.Separando as variáveis preditoras vs. target

In [54]:
X_credit = credit_data.drop(["clientid", "default"], axis=1)
y_credit = credit_data.iloc[:,-1]

In [55]:
X_credit.head(3)

,income,age,loan
0,66155.925095,59,8106.532131
1,34415.153966,48,6564.745018
2,57317.170063,63,8020.953296


In [56]:
y_credit.head()

0    0
1    0
2    0
3    0
4    1
Name: default, dtype: int64

----

### 3.Transformação dos dados: escalonamento
O objetivo desse tratamento é deixar os dados na mesma escala. Isso porque alguns algoritmos (como aqueles baseados em distância como o KNN) podem atribuir pesos diferentes para variáveis com dimensões diferentes. E esse não é o nosso objetivo. Para escalonar os dados, vamos usar a técnica da padronização:

In [57]:
# mínimos e máximos antes do escalonamento
X_credit.describe()

,income,age,loan
count,2000.000000,2000.000000,2000.000000
mean,45331.600018,40.425500,4444.369695
std,14326.327119,13.262594,3045.410024
min,20014.489470,18.000000,1.377630
25%,32796.459717,29.000000,1939.708847
50%,45789.117313,41.000000,3974.719419
75%,57791.281668,52.000000,6432.410625
max,69995.685578,63.000000,13766.051239


In [58]:
scaler_credit = StandardScaler()
X_credit = scaler_credit.fit_transform(X_credit)
pd.DataFrame(X_credit).describe()

,0,1,2
count,2.000000e+03,2.000000e+03,2.000000e+03
mean,4.718448e-17,4.596323e-17,1.255662e-16
std,1.000250e+00,1.000250e+00,1.000250e+00
min,-1.767616e+00,-1.691306e+00,-1.459279e+00
25%,-8.751912e-01,-8.616985e-01,-8.226437e-01
50%,3.194341e-02,4.332815e-02,-1.542543e-01
75%,8.699227e-01,8.729359e-01,6.529624e-01
max,1.722022e+00,1.702544e+00,3.061661e+00


### 4.Separação das bases em treino e teste

Para criarmos a base de treino que servirá para os algoritmos aprenderem os padrões nos dados históricos e também a base de teste para avaliar a acurácia do modelo treinado, vamos usar a proporção de 75% treino e 25% teste:

In [61]:
X_credit_train, X_credit_test,y_credit_train, y_credit_test = train_test_split(X_credit, 
                                                                                 y_credit,
                                                                                test_size=0.25,
                                                                                random_state=42)

O parâmetro "random_state" recebendo um valor inteiro cria uma semente para a função que gera as divisões aleatórias da base e garante que os resultados sejam reprodutíveis. Ou seja, que sempre tenhamos os mesmos valores separados para treino e teste. Checando o tamanho das bases:

In [63]:
X_credit_train.shape, X_credit_test.shape

((1500, 3), (500, 3))

In [64]:
y_credit_train.shape, y_credit_test.shape

((1500,), (500,))

### 5.Exportando os dados para usarmos em diferentes algoritmos:

In [67]:
with open("credit.pkl", mode="wb") as f:
    pickle.dump([X_credit_train, y_credit_train, X_credit_test, y_credit_test], f)